In [0]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
from scipy import stats
from datetime import datetime
from scipy.stats import pearsonr
import re
from sklearn.decomposition import PCA

import seaborn as sns

from scipy import sparse
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 92kB 2.4MB/s 


In [0]:
data = pd.read_csv('/content/gdrive/My Drive/DM-group/train.zip', compression='zip', header=0, sep=',', quotechar='"')


In [1]:
data = data.sample(frac=0.1, replace='False')
data.reset_index(inplace=True, drop=True)
data.info()

In [0]:
# we delete the rows with srch_ci and srch_co NaN
(data.srch_ci[data["srch_ci"].isnull()].index)
indexes = (data.srch_ci[data["srch_co"].isnull()].index)
data = data.drop(indexes)

In [0]:
def is_weekend(ci, co):
    ci_wd = ci.weekday()
    co_wd = co.weekday()
    li = list(range(ci_wd, co_wd+1))
    if 5 in li and 6 in li:
        return 1
    else:
        return 0

In [0]:
# Transform dates into datetime to extract month and day
data["date_time"] = pd.to_datetime(data["date_time"], errors="coerce")
data["booking_month"] = data.date_time.apply(lambda x: x.month)
data["booking_day"] = data.date_time.apply(lambda x: x.day)


data["srch_ci"] = pd.to_datetime(data["srch_ci"], errors="coerce")
data["srch_co"] = pd.to_datetime(data["srch_co"], errors="coerce")

data["ci_month"] = data.srch_ci.apply(lambda x: x.month)
data["co_month"] = data.srch_co.apply(lambda x: x.month)
data["ci_day"] = data.srch_ci.apply(lambda x: x.day)
data["co_day"] = data.srch_co.apply(lambda x: x.day)

# Create new features
data["duration"] = data.apply(lambda row: (row.srch_co - row.srch_ci).days, axis=1)
data["remaining_days"] = data.apply(lambda x: (x.srch_ci - x.date_time).days, axis=1 )
data["is_weekend"] = data.apply(lambda row: is_weekend(row.srch_ci, row.srch_co), axis=1)

# Delete original date features
data = data.drop(columns=["date_time", "srch_ci", "srch_co"])

In [ ]:
data.to_csv("1_training_preprocessed.csv", index=False)

### IMPUTING MISSING VALUES OF ORIG_DEST_DISTANCE

In [0]:
cols2keep=['site_name','posa_continent','user_location_country',
 'user_location_region',
 'user_location_city',
 'orig_destination_distance',
 'hotel_continent',
 'hotel_country']

from sklearn.experimental import enable_iterative_imputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import IterativeImputer

#Define predictor
predictor = RandomForestRegressor(bootstrap= True,
 max_depth= 100,
 max_features= 3,
 min_samples_leaf= 3,
 min_samples_split= 8,
 n_estimators= 300)

#Define imputation function
imp = IterativeImputer(max_iter=10, estimator=predictor)


In [2]:
#Fit the model which will be used to predict missing values later and fill the missing values
chunksize = 100000
count = 0
train = pd.read_csv(base_dir + "1_training_preprocessed.csv", chunksize=chunksize, usecols=cols2keep)
for chunk in train:
    imp.fit(chunk)
    count = count + chunksize
    print('%d rows trained' % count)

In [3]:
#Predict the missing values
full=pd.DataFrame()
count = 0
train = pd.read_csv(base_dir + "10_training_preprocessed.csv", chunksize=chunksize, usecols=cols2keep)
for chunk in train:
  chunk=pd.DataFrame(np.round(imp.transform(chunk), 4))
  full=full.append(chunk)
  count = count + chunksize
  print('%d rows predicted' % count)

In [0]:
full.shape

(3762223, 8)

In [0]:
#Check if missing values are left
full[full.iloc[:,7].isna()]

,0,1,2,3,4,5,6,7


In [0]:
train_again= pd.read_csv(base_dir + "1_training_preprocessed.csv")
train_again.reset_index(inplace=True)

In [0]:
#Access the result of imputation
#Create a copy of the training dataset, make existing distance values missing 
test=train_again.copy()
test=test[['site_name','posa_continent','user_location_country',
 'user_location_region',
 'user_location_city',
 'orig_destination_distance',
 'hotel_continent',
 'hotel_country']]
test.orig_destination_distance=None

In [0]:
#See what would have been predicted for non missing distances
full_test=pd.DataFrame(np.round(imp.transform(test),4))

In [0]:
#For those distances that were not missing, merge original destinations and predicted destinations
a=pd.DataFrame(full.iloc[:,5])
b=pd.DataFrame(full_test.iloc[:,5])
c=a.merge(b, left_index=True, right_index=True)
d=c[train_again.orig_destination_distance.isnull()==False]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  after removing the cwd from sys.path.


In [0]:
#Merge the imputed data with the one original dataset
train_again=train_again.reset_index(drop=True)

In [0]:
imputed_distances = (full.iloc[:,5]).reset_index(drop=True)
data["orig_destination_distance"] = imputed_distances

In [0]:
data.to_csv("1_training_imputed.csv", index=False)

## Reduce dimensionality of destination latent features (PCA)

In [ ]:
dest = pd.read_csv(base_dir + "destinations.csv")

In [ ]:
X = dest.iloc[:,1:]
pca = PCA().fit(X)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
pca = PCA(n_components=40)
X = pca.fit_transform(X)
X.shape

In [ ]:
pd.DataFrame(X).to_csv("destinations_pca.csv")

## Merge latent destination features with original ones

In [5]:
dest = pd.read_csv(base_dir + "destinations_pca.csv")

In [0]:
data_tot = data.merge(dest, on='srch_destination_id', how='left')
data_tot.head()

,site_name,posa_continent,user_location_country,user_location_region,user_location_city,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_adults_cnt,srch_children_cnt,srch_rm_cnt,srch_destination_id,srch_destination_type_id,is_booking,cnt,hotel_continent,hotel_country,hotel_market,hotel_cluster,booking_month,booking_day,ci_month,co_month,ci_day,co_day,duration,remaining_days,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,2,3,66,337,1794,1610.0578,44129,0,1,0,2,1,1,12257,6,0,1,4,51,1509,51,7,10,8.0,8.0,6.0,10.0,4.0,26.0,1.191585,0.744867,-0.469529,0.047483,0.093242,0.264022,0.170370,-0.116922,-0.039931,-0.082221,0.173404,-0.173956,-0.165731,-0.078212,-0.001044,-0.040452,-0.022311,-0.024057,-0.009754,-0.023704,-0.014160,-0.088971,-0.061519,-0.059767,-0.031309,0.056486,0.075728,-0.102674,-0.005740,0.009403,-0.034844,-0.001772,-0.100779,0.058327,-0.045351,-0.000418,-0.095607,-0.018624,0.075043,-0.024062
1,11,3,205,330,34157,1582.3503,636659,0,1,5,2,0,1,12008,1,0,1,2,50,686,30,10,18,2.0,3.0,21.0,7.0,14.0,125.0,0.783531,0.158181,0.158007,-0.104123,-0.103367,0.018827,-0.108281,-0.033807,-0.012767,0.018509,0.118418,0.025077,0.079333,0.018046,0.119741,0.023670,-0.077435,-0.192493,0.143602,0.115970,-0.098889,0.095615,0.199725,0.181496,-0.047639,-0.172091,-0.021685,-0.028080,0.086374,0.060094,0.004900,-0.019996,-0.020920,-0.056257,-0.026258,-0.034102,0.033088,0.007355,-0.013114,0.031350
2,2,3,66,153,25263,5819.9701,205466,0,0,2,2,0,1,8275,1,0,1,4,196,2027,61,8,5,8.0,8.0,19.0,20.0,1.0,13.0,0.345613,0.317985,-0.501484,0.058383,0.123818,0.004131,-0.210061,0.062186,-0.111506,0.006034,0.068205,-0.204644,-0.123717,-0.013068,-0.007315,-0.022657,-0.169726,-0.027370,-0.046971,-0.060071,-0.030012,0.104057,0.013033,0.014581,-0.100116,0.006442,0.045041,-0.050332,0.074114,0.019229,-0.108631,0.030677,0.013691,0.113369,0.121182,-0.063018,0.067872,-0.010622,-0.132900,0.071599
3,24,2,3,70,26750,777.6096,242199,0,1,7,2,0,1,8253,1,0,1,6,70,19,57,2,19,3.0,3.0,3.0,8.0,5.0,11.0,0.917121,-0.456688,-0.380253,0.217810,-0.047301,0.168425,-0.142490,0.095163,0.032296,-0.019697,-0.060303,-0.114995,-0.092906,0.059910,0.044069,-0.034655,-0.079383,-0.098699,0.095721,-0.050602,0.052770,0.010083,-0.056300,-0.045835,0.053675,-0.051244,0.043089,-0.033936,0.050093,0.063033,0.042602,-0.030699,0.044762,-0.031511,-0.027542,0.011744,0.071806,-0.049978,0.029836,0.019557
4,2,3,66,226,37619,5640.1696,166051,0,1,9,2,0,1,11917,1,0,2,0,140,59,92,12,26,9.0,9.0,5.0,11.0,6.0,252.0,-0.022336,0.467553,-0.026043,0.197194,-0.037272,-0.050407,0.014176,0.103557,0.199025,-0.013037,-0.030791,-0.016361,0.059263,-0.168816,0.059814,-0.106974,0.152623,-0.066421,-0.119054,0.036888,-0.089477,-0.039241,0.019853,-0.062358,0.065853,-0.054756,-0.061903,0.011048,-0.035871,0.064005,0.142337,-0.102710,-0.102408,-0.099054,-0.031315,0.073065,0.007605,0.070549,-0.053587,-0.019694


In [0]:
data_tot = data_tot.drop(columns=["srch_destination_id", "srch_destination_type_id"])

In [0]:
# data_tot = data_tot.drop(columns=["srch_destination_id", "srch_destination_type_id"])
data_tot.columns = ['site_name', 'posa_continent', 'user_location_country',
       'user_location_region', 'user_location_city',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'is_booking', 'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster', 'booking_month', 'booking_day', 'ci_month', 'co_month', 'ci_day',
       'co_day', 'duration', 'remaining_days', 'dest0', 'dest1', 'dest2', 'dest3', 'dest4', 'dest5', 'dest6',
       'dest7', 'dest8', 'dest9', 'dest10', 'dest11', 'dest12', 'dest13', 'dest14', 'dest15', 'dest16', 'dest17', 'dest18',
       'dest19', 'dest20', 'dest21', 'dest22', 'dest23', 'dest24', 'dest25', 'dest26', 'dest27', 'dest28', 'dest29', 'dest30',
       'dest31', 'dest32', 'dest33', 'dest34', 'dest35', 'dest36', 'dest37', 'dest38', 'dest39']

In [0]:
data_tot.shape

(374623, 68)

In [0]:
indexess = (data_tot[data_tot["dest0"].isna()]).index
data_tot = data_tot.drop(indexess)
data_tot.shape

(374623, 68)

In [ ]:
data_tot.to_csv("1_training_final.csv", index=False)